[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sparks-baird/self-driving-lab-demo/blob/main/notebooks/4.0-paho-mqtt-colab-sdl-demo.ipynb)

# Internet of Things-style communication between Pico W and some other device

The other device could be your laptop, a Google Colab script, etc.

In [1]:
#@markdown This cell checks if you're running in Google Colab or not and assigns it to `IN_COLAB`.

from os import path
try:
  import google.colab
  IN_COLAB = True
  base_dir = "/content/drive/MyDrive/"
except:
  IN_COLAB = False
  base_dir = path.join("data", "external")
  

In [2]:
if IN_COLAB:
  %pip install paho-mqtt

## Pico ID

Load a unique identifier for your Pico device from a `pico_id_secret.py` file that you store in your Google Drive (see below) or current directory (`.`), or specify it manually by setting the `PICO_ID` variable in the form field. If you don't know what the ID is, you can get it by running the following in the Pico REPL (i.e. MicroPython shell, e.g. in Thonny):
```python
from machine import unique_id
from ubinascii import hexlify
my_id = hexlify(unique_id()).decode()
print(my_id)
```

For your convenience, this is done already in [`main.py`](https://github.com/sparks-baird/self-driving-lab-demo/blob/06ededd312087a815b5c0ac8098e82db9c5730c8/src/public_mqtt_sdl_demo/main.py) so you need only run this file once the Pico W has been properly set up and it will be displayed in the terminal.

The file named `pico_id_secret.py` placed directly in your `MyDrive` folder or in your current directory would look like the following:
```python
PICO_ID = "a123b1234azed"
```

In [3]:
#@markdown No need to enter this manually if you've already saved the appropriate file to your Google Drive. The hidden code in this cell will take care of it.
import sys
secret_name = "PICO_ID"
if IN_COLAB:
  try:
    from google.colab import drive
    drive.mount('/content/drive')
    sys.path.append('/content/drive/MyDrive/')
    from pico_id_secret import PICO_ID
  except Exception as e:
    print(e)
    PICO_ID = "test" #@param {type:"string"}
    if PICO_ID == "":
      print(f"Couldn't load {secret_name}, and user-input {secret_name} is also empty.")
    else:
      print(f"defaulting to user-input PICO_ID: {PICO_ID}")
else:
  from public_mqtt_sdl_demo.secrets import PICO_ID

In [4]:
"""https://www.steves-internet-guide.com/receiving-messages-mqtt-python-clientq=Queue()"""
import json
from uuid import uuid4
import paho.mqtt.client as mqtt
from queue import Queue
from time import time

sensor_data_queue = Queue()
timeout = 30 # seconds

def on_message(client, userdata, msg):
    sensor_data_queue.put(json.loads(msg.payload))


def observe_sensor_data(
    R, G, B, pico_id=None, session_id=None, hostname="test.mosquitto.org"
):
    if session_id is None:
        session_id = str(uuid4())

    experiment_id = str(uuid4())

    prefix = f"sdl-demo/picow/{pico_id}/"
    neopixel_topic = prefix + "GPIO/28"
    sensor_topic = prefix + "as7341/"

    # The callback for when the client receives a CONNACK response from the server.
    def on_connect(client, userdata, flags, rc):
        if rc != 0:
            print("Connected with result code " + str(rc))
        # Subscribing in on_connect() means that if we lose the connection and
        # reconnect then subscriptions will be renewed.
        client.subscribe(sensor_topic, qos=2)

    client = mqtt.Client()  # create new instance
    client.on_connect = on_connect
    client.on_message = on_message
    client.connect(hostname)  # connect to broker
    client.subscribe(sensor_topic, qos=1)

    # ensures double quotes for JSON compatiblity
    payload = json.dumps(
        dict(
            R=int(R),
            G=int(G),
            B=int(B),
            _session_id=session_id,
            _experiment_id=experiment_id,
        )
    )
    client.publish(neopixel_topic, payload, qos=1)

    client.loop_start()
    while True:
        sensor_data = sensor_data_queue.get(timeout)
        inp = sensor_data["_input_message"]
        if inp["_session_id"] == session_id and inp["_experiment_id"] == experiment_id:
            assert inp["R"] == R, "red value mismatch"
            assert inp["G"] == G, "green value mismatch"
            assert inp["B"] == B, "blue value mismatch"
            client.loop_stop()
            sensor_data.pop("_input_message") # remove the input message
            return sensor_data


print(observe_sensor_data(5, 10, 15, pico_id=PICO_ID))
print(observe_sensor_data(20, 30, 25, pico_id=PICO_ID))
print(observe_sensor_data(0, 0, 0, pico_id=PICO_ID))


{'ch470': 4390, 'ch670': 2044, 'ch550': 1287, 'ch410': 613, 'ch440': 2378, '_input_message': {'_session_id': '8518e95c-049a-47a9-9e6c-6a9ea8dc4d20', 'R': 5, 'B': 15, 'G': 10, '_experiment_id': 'a94236b3-43df-4c6b-a2b7-34d890ca1c29'}, 'ch583': 1252, 'ch510': 1441, 'ch620': 1490}
{'ch470': 6565, 'ch670': 2211, 'ch550': 1599, 'ch410': 737, 'ch440': 4125, '_input_message': {'_session_id': 'de676c55-2690-4079-8abe-fbcbb462956a', 'R': 20, 'B': 25, 'G': 30, '_experiment_id': '458358c1-9697-4595-b30e-733325570b5e'}, 'ch583': 2044, 'ch510': 3881, 'ch620': 3072}
{'ch470': 3470, 'ch670': 1964, 'ch550': 1212, 'ch410': 558, 'ch440': 1076, '_input_message': {'_session_id': 'e6489fcb-fc15-4e50-b6a5-9402624221f6', 'R': 0, 'B': 0, 'G': 0, '_experiment_id': 'db7cc63b-56b5-4547-bda2-2f81349fb3b1'}, 'ch583': 1107, 'ch510': 1000, 'ch620': 1215}
